## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df=pd.read_csv('/content/drive/MyDrive/3 - DLCVNLP Projects/Fake_News_Classifier_Using - LSTM & Bidirectional LSTM/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.5.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [16]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3849, 1077, 819, 2339, 1283, 891, 1473, 1856, 287, 1629],
 [2372, 1426, 480, 3398, 317, 4820, 1576],
 [4086, 4719, 3929, 500],
 [3910, 3838, 4955, 2042, 1924, 917],
 [4790, 317, 1819, 3433, 1873, 4276, 317, 4402, 2554, 563],
 [4486,
  3918,
  2297,
  2596,
  1655,
  1517,
  2318,
  4661,
  1094,
  2384,
  3503,
  275,
  4386,
  831,
  1576],
 [1743, 290, 4234, 4792, 1854, 1706, 4037, 4892, 859, 3447, 943],
 [1606, 1362, 354, 4614, 4667, 1620, 1517, 4081, 859, 3447, 943],
 [2142, 3211, 4255, 3105, 4316, 2033, 2562, 1067, 1517, 3006],
 [4583, 2169, 3559, 448, 915, 4362, 2870, 2953],
 [71, 416, 3295, 2055, 2198, 2156, 1153, 1301, 2538, 4218, 1810],
 [2042, 2390, 1283, 2033, 1517, 4667],
 [4301, 2561, 87, 678, 3528, 2207, 4046, 1563, 1614],
 [2126, 2714, 933, 2948, 2976, 730, 3612, 859, 3447, 943],
 [587, 2932, 2475, 2988, 1238, 859, 3447, 943],
 [4732, 1201, 3894, 1317, 1431, 2097, 635, 1217, 3106, 522],
 [1830, 1014, 1426],
 [724, 1112, 188, 3405, 1517, 2948, 3861, 1576],
 [1177, 294, 

### Embedding Representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1856  287 1629]
 [   0    0    0 ...  317 4820 1576]
 [   0    0    0 ... 4719 3929  500]
 ...
 [   0    0    0 ...  859 3447  943]
 [   0    0    0 ...  836 3164 3142]
 [   0    0    0 ...  888 3846  971]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3849,
       1077,  819, 2339, 1283,  891, 1473, 1856,  287, 1629], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape

(18285, (18285,))

In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [48]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 38ms/step - loss: 0.3557 - accuracy: 0.8235 - val_loss: 0.2143 - val_accuracy: 0.9137
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1532 - accuracy: 0.9398 - val_loss: 0.2130 - val_accuracy: 0.9216
Epoch 3/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1119 - accuracy: 0.9577 - val_loss: 0.2015 - val_accuracy: 0.9191
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0771 - accuracy: 0.9731 - val_loss: 0.2153 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0577 - accuracy: 0.9802 - val_loss: 0.2747 - val_accuracy: 0.9140
Epoch 6/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0349 - accuracy: 0.9883 - val_loss: 0.3219 - val_accuracy: 0.9104
Epoch 7/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0288 - accuracy: 0.9900 - val_loss: 0.3915 - val_accuracy: 0.908

### Adding Dropout 

In [74]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [75]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
confusion_matrix(y_test,y_pred)

array([[2509,  910],
       [1130, 1486]])

In [78]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6619718309859155